<a href="https://colab.research.google.com/github/techYamasaki/TopicModelYouTube/blob/master/save_livedoor_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# livedoorコーパスの保存

In [0]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar xvf ldcc-20140209.tar.gz

In [0]:
!pip install sudachipy
!pip install pyLDAvis
!pip install SudachiPy
!pip install https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/sudachi/SudachiDict_core-20190927.tar.gz
from gensim import corpora, models
from os import listdir, path
from sudachipy import tokenizer, dictionary
from collections import Counter
from wordcloud import WordCloud
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim

In [0]:
class SudachiAnalizer():
    
    def get_token(self, source) :
        
        tokenizer_obj = dictionary.Dictionary().create()

        mode = tokenizer.Tokenizer.SplitMode.C
        result = tokenizer_obj.tokenize(source, mode)

        word_list = []
        for mrph in result:
            if not (mrph == ""):
                norm_word = mrph.normalized_form()
                hinsi = mrph.part_of_speech()[0] 

                # 単語の正規表現が特定の品詞の場合のみ採用する
                if hinsi in  ["名詞", "動詞", "形容詞"]:
                    word = norm_word
                    word_list.append(word)

        return word_list

In [0]:
docs = []
PATH = "text/"

sudachi = SudachiAnalizer()
#  pathの中のdir(txt以外)をlistにして返す
def corpus_subdirs(path):
    subdirs = []
    for x in listdir(path):
        if not x.endswith('.txt'):
            subdirs.append(x)
    return subdirs

# pathの中のファイルをlistにして返す
def corpus_filenames(path):
    labels = [] # *.txt
    for y in listdir(path):
        if not y.startswith('LICENSE'):
            labels.append(y)
    return labels

for dir in corpus_subdirs(PATH):
    for file in corpus_filenames(PATH+dir):
        corpus_data = open(path.join(PATH + dir + "/" + file), "r")
        source = corpus_data.read()
        token = sudachi.get_token(source)
        corpus_data = {"name" : file, "tag" : dir, "token" : token}
        docs.append(corpus_data)

In [0]:
with open('corpus.pkl', 'wb') as temp:
  pickle.dump(docs, temp)